# Índice Invertido y Modelo Booleano

In [ ]:
import nltk
import regex as re
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')

stemmer = SnowballStemmer('spanish')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### P1 - Tokenizar y eliminar stopwords

In [ ]:
with open("stoplist.txt", encoding="latin1") as file:
    stoplist = [line.rstrip().lower() for line in file]

def preProcesamiento(texto,stemming=True):
    words = []
    # 1- convertir a minusculas
    texto = texto.lower()
    # 2- eliminar signos con regex
    texto = re.sub(r'[^a-zA-Z_À-ÿ]', ' ', texto) #quité los números
    # 3- tokenizar
    words = nltk.word_tokenize(texto, language='spanish')
    # 3- eliminar stopwords
    words = [word for word in words if word not in stoplist]
    # 4- Aplicar reduccion de palabras (stemming)
    if stemming:        
        words = [stemmer.stem(word) for word in words]
    return words

# procesar los textos
preProcesamiento("hola a todos! aquí casual programando 91293 9199391 ") #antes los numero no eran quitados

['hol', 'aqu', 'casual', 'program']

### P2 - Construir Índice Invertido

In [ ]:
import csv
def indexar(filename, top_k):    
    indice = {}
    with open(filename, encoding="utf-8") as File:
        csv_reader = csv.reader(File, delimiter=',')
        next(csv_reader) # skip header
        for line in csv_reader:
            # 1- preprocesar
            words = preProcesamiento(line[1]) #cada body de cadalinea es un texto
            
        # 2- Filtrar top-k palabras más frecuentes
        dict_words = {}
        for w in words:
            if w in dict_words:
                dict_words[w] += 1
            else:
                dict_words[w] = 1
        #ordeno las las top_k palabras más comunes (ya preprocesadas)
        sorted_words = sorted(dict_words, key=dict_words.get, reverse=True)[:top_k]

        #me ubico al comienzo del archivo para leer nuevamente
        File.seek(0)
        next(csv_reader) #saltar header

        # 3- agregar los keywords al indice 
        i=1
        for line in csv_reader: #recorro cada texto nuevamente
            for word in sorted_words:
                if word in line[1]:
                    if word in indice:
                        indice[word].append(i)
                    else:
                        indice[word] = [i]
            i+=1

    return indice

# indexar
top_k = 500
Indice = indexar("news_es.csv", top_k)
Indice = (sorted(Indice.items()))
#print(Indice) #solo para ver si funca

# Guardar indice con el formato del pdf
with open("indices_invertidos_top500.txt", "w") as file: #archivos para 500,1000 y 2000
    for item in Indice:
        file.write(f'{item[0]}: {item[1]}\n')

### P3 - Aplicar consultas

In [ ]:
import ast
def AND(list1, list2):
    n = 0 
    m = 0
    query = []
    while n < len(list1) and m < len(list2): 
        if list1[n] == list2[m]:
            query.append(list1[n])
            n += 1
            m += 1
        else: #sumo el menor
            if n < m:
                n += 1
            else:
                m += 1
    return query

def OR(list1, list2):
    n = 0
    m = 0
    query = []
    while n < len(list1) or m < len(list2):
        if list1[n] == list2[m]:
            query.append(list1[n])
            n += 1
            m += 1
        elif list1[n] < list2[m]:
            query.append(list1[n])
            n += 1
        else:
            query.append(list2[m])
            m += 1
    return query

def AND_NOT(list1, list2):
    # Convert list2 to a set for O(1) average time complexity on membership checks
    set2 = set(list2)
    query = [item for item in list1 if item not in set2]
    return query


def L(word):
    query = []
    with open("indices_invertidos_top2000.txt", encoding="latin1") as file:
        for line in file:
            word, indices = line.split(':')
            word = word.strip() 
            indices = indices.strip()
            query = ast.literal_eval(indices)  
    return query

import csv

def mostrarResultados(result):
    with open("news_es.csv", encoding="latin1") as file:
        csv_reader = csv.reader(file, delimiter=',')
        lines = list(csv_reader) 
        for i in result:
            if i < len(lines):  
                line = lines[i]  
                if len(line) > 1:  
                    print(line[1][:75], "...") 

# Example usage
result = [1, 3, 5]  # Indices of rows to display
mostrarResultados(result)


Durante el foro La banca articulador empresarial para el desarrollo sosteni ...
En una industria histÃ³ricamente masculina como lo es la aviaciÃ³n Viva pre ...
Ayer en Cartagena se dio inicio a la versiÃ³n nÃºmero 56 de la ConvenciÃ³n  ...


### Consulta AND

In [ ]:

result = AND(L("educación"), L("aprendizaje"))
print("educación & aprendizaje :");
print(mostrarResultados(result))
#mostrar un poquito del texto de la noticia o el titulo


La marca personal ha dejado de ser solo para emprendedores y se ha converti ...
El Ministerio de EnergÃ­a y Minas Minem revelÃ³ que esta semana emitirÃ­a e ...
BBVA Research ha actualizado al alza su previsiÃ³n de crecimiento para el c ...
Delta Air Lines y Latam Airlines Group y sus filiales en Brasil Colombia Pe ...
Visa anunciÃ³ una alianza con Minsait Payments la filial de medios de pago  ...
El Departamento de Justicia presentarÃ¡ una propuesta al Congreso el miÃ©rc ...
El auge de las criptomonedas, especialmente en regiones como AmÃ©rica Latin ...
MÃ¡s de 600 jÃ³venes de nueve nacionalidades diferentes han participado en  ...
El aprendizaje automÃ¡tico, el metaverso y los chatbots son algunas de las  ...
La voz entusiasta de Odile describe con admiraciÃ³n la vida y el pensamient ...
El ComitÃ© Federal de Mercado Abierto FOMC por sus siglas en inglÃ©s rompiÃ ...
Los clientes de BBVA del sector de la hostelerÃ­a en EspaÃ±a que dispongan  ...
Los expertos en finanzas personales reco

### Consulta OR

In [ ]:
result = OR(L("formal"), L("marca"))
print("formal | marca :");
print(mostrarResultados(result))

La marca personal ha dejado de ser solo para emprendedores y se ha converti ...
El Ministerio de EnergÃ­a y Minas Minem revelÃ³ que esta semana emitirÃ­a e ...
BBVA Research ha actualizado al alza su previsiÃ³n de crecimiento para el c ...
Delta Air Lines y Latam Airlines Group y sus filiales en Brasil Colombia Pe ...
Visa anunciÃ³ una alianza con Minsait Payments la filial de medios de pago  ...
El Departamento de Justicia presentarÃ¡ una propuesta al Congreso el miÃ©rc ...
El auge de las criptomonedas, especialmente en regiones como AmÃ©rica Latin ...
MÃ¡s de 600 jÃ³venes de nueve nacionalidades diferentes han participado en  ...
El aprendizaje automÃ¡tico, el metaverso y los chatbots son algunas de las  ...
La voz entusiasta de Odile describe con admiraciÃ³n la vida y el pensamient ...
El ComitÃ© Federal de Mercado Abierto FOMC por sus siglas en inglÃ©s rompiÃ ...
Los clientes de BBVA del sector de la hostelerÃ­a en EspaÃ±a que dispongan  ...
Los expertos en finanzas personales reco

### Consulta AND_NOT

In [ ]:
result = AND_NOT(L("necesito"), L("mundo"))
print("necesito &~ mundo :");
print(mostrarResultados(result))

necesito &~ mundo :
None


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c33856d2-92e2-40a2-8b39-4bc95d5021ae' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>